In [1]:
# cross section preprocessing based on peaks of elevation

In [2]:
# given cross section, find all peaks in elevation
# can also use this to compute subbasin valley depth - as elevation difference between thalweg and ridges, atleast in 2d

In [170]:
from slopes.__main__ import process_topography, load_input, setup_wbt, smooth_flowlines
from slopes.profile.preprocess_profile import preprocess_profiles
from slopes.profile.network_xsections import network_xsections
from slopes.utils import observe_values

import matplotlib.pyplot as plt

In [118]:
wbt = setup_wbt("../working_dir/")

In [119]:
dem, flowlines = load_input("../data/input/dem.tif", "../data/input/flowlines.shp")

In [120]:
dataset, aligned_flowlines = process_topography(dem, flowlines, wbt, sigma=5)
smoothed = smooth_flowlines(aligned_flowlines)

In [171]:
xsections = network_xsections(smoothed, 10, 300, 3)

In [172]:
values = observe_values(xsections, dataset[['dem', 'slope', 'curvature', 'hand', 'hillslope', 'flow_path', 'subbasin']])

In [173]:
processed = preprocess_profiles(values, min_hand_jump=15, ratio=5, min_distance=5, min_peak_prominence=20)

/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/slopes-mqMb1mKH-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/slopes-mqMb1mKH-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/slopes-mqMb1mKH-py3.10/

In [174]:
values.to_file('xsections.shp')
processed.to_file('processed.shp')

In [136]:
dataset['curvature'].rio.to_raster('curv.tif')

In [152]:
xsections

,geom,pointID,streamID,xsID,alpha,dem,slope,curvature,hand,hillslope,flow_path,subbasin
0,POINT (393573.463 3791304.16),0,1.0,0,-300,944.783589,31.744574,0.221380,86.297546,1.0,NaN,1.0
1,POINT (393576.383 3791303.473),1,1.0,0,-297,945.352776,30.257847,0.230204,86.826660,1.0,NaN,1.0
2,POINT (393579.303 3791302.786),2,1.0,0,-294,945.853309,28.437632,0.233769,87.224854,1.0,NaN,1.0
3,POINT (393582.224 3791302.1),3,1.0,0,-291,945.853309,28.437632,0.233769,87.224854,1.0,NaN,1.0
4,POINT (393585.144 3791301.413),4,1.0,0,-288,946.327328,27.258038,0.227372,87.586243,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
98686,POINT (393850.079 3790500.729),98686,14.0,16,288,871.112667,31.233234,-0.001306,105.350830,1.0,NaN,10.0
98687,POINT (393847.169 3790500.002),98687,14.0,16,291,869.351314,30.893967,-0.001494,103.654724,1.0,NaN,10.0
98688,POINT (393844.258 3790499.275),98688,14.0,16,294,867.741792,30.443996,-0.001498,101.916016,1.0,NaN,10.0
98689,POINT (393841.347 3790498.548),98689,14.0,16,297,866.150470,29.946405,-0.001669,100.376221,1.0,NaN,10.0


In [166]:
import rioxarray as rxr

raster = rxr.open_rasterio("../data/input/dem.tif", masked=True)
raster.name = 'dem'
raster.to_dataframe().reset_index()

,band,y,x,spatial_ref,dem
0,1,3791360.5,393625.5,0,927.526123
1,1,3791360.5,393626.5,0,926.605469
2,1,3791360.5,393627.5,0,925.460815
3,1,3791360.5,393628.5,0,924.557373
4,1,3791360.5,393629.5,0,924.383545
...,...,...,...,...,...
1885015,1,3790293.5,395385.5,0,1207.093872
1885016,1,3790293.5,395386.5,0,1207.421875
1885017,1,3790293.5,395387.5,0,1207.828491
1885018,1,3790293.5,395388.5,0,1208.195679
